In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import acquire
print("Scucess")

Scucess


# Pre Zillow
This notebook is used to prepare the zillow_wrangle.py function by performing basic operations to see things like value counts on columns, detect outliers, etc.

In [2]:
help(acquire)

Help on module acquire:

NAME
    acquire

FUNCTIONS
    columns_data(df)
        Takes in a data frame and returns the number of rows and percentage of rows missing per column
    
    get_connection(db, user='darden_1027', host='157.230.209.171', password='nIymKg6jJmIPe6gQxik57EkY3Fl9kmvl')
        Returns a formatted url with login credentials to access data on a SQL database.
    
    load_zillow_data()
        This function acquires the zillow dataset from a SQL Database.
        It returns the zillow dataset as a Pandas DataFrame.
        
        A local copy will be created as a csv file in the current directory for future use.
    
    row_data(df)
        Takes in a data frame and returns the range of columns with missing data, 
        what precentage of missing columns that represents,
        and how many rows have that number of missing columns
    
    summarize(df)
        Prints the basic summary statistics of a data frame

FILE
    c:\users\anthony\documents\data_scie

In [3]:
df = acquire.load_zillow_data()
df.shape

C:\Users\anthony\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3337: DtypeWarning: Columns (69) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


(77381, 70)

In [4]:
acquire.row_data(df)

,num_cols_missing,num_rows,pct_cols_missing
0,23,2,0.328571
1,24,13,0.342857
2,25,24,0.357143
3,26,65,0.371429
4,27,316,0.385714
5,28,455,0.400000
6,29,5270,0.414286
7,30,3455,0.428571
8,31,9891,0.442857
9,32,12579,0.457143


In [5]:
acquire.columns_data(df)

,columns,num_rows_missing,pct_rows_missing
0,buildingclasstypeid,77366,0.999806
1,typeconstructiontypeid,77159,0.997131
2,storytypeid,77331,0.999354
3,propertylandusetypeid,0,0.000000
4,heatingorsystemtypeid,27941,0.361083
...,...,...,...
65,heatingorsystemdesc,27941,0.361083
66,propertylandusedesc,0,0.000000
67,storydesc,77331,0.999354
68,typeconstructiondesc,77159,0.997131


In [6]:
df.poolcnt.value_counts()

1.0    16137
Name: poolcnt, dtype: int64

In [7]:
df.basementsqft.isnull().sum()


77331

In [8]:
df.storydesc.unique()

array([nan, 'Basement'], dtype=object)

In [9]:
df.heatingorsystemdesc.unique()

array([nan, 'Central', 'Floor/Wall', 'Forced air', 'Yes', 'None',
       'Radiant', 'Solar', 'Baseboard', 'Gravity', 'Heat Pump'],
      dtype=object)

In [10]:
print(df.fireplacecnt.isnull().sum())
print(df.fireplaceflag.isnull().sum())

69105
77209


In [11]:
((df['fireplaceflag'] == 1) == (df.fireplacecnt.isnull())).mean() * 100

10.917408666210052

In [12]:
df.hashottuborspa.unique()

array([nan,  1.])

In [13]:
df['propertylandusedesc'].loc[df['unitcnt'] == 1].value_counts()

Single Family Residential                   33734
Condominium                                 11494
Planned Unit Development                     1937
Mobile Home                                    53
Cooperative                                    29
Duplex (2 Units, Any Combination)              20
Residential General                            10
Commercial/Office/Residential Mixed Used        8
Townhouse                                       6
Triplex (3 Units, Any Combination)              2
Name: propertylandusedesc, dtype: int64

In [14]:
df['fips'].loc[df['unitcnt'] == 1].value_counts()

6037.0    47282
6059.0       10
6111.0        1
Name: fips, dtype: int64

In [15]:
def prep_zillow():
    '''
    Loads zillow data and then fliters it where propertylandusetypeid is 260, 261, 262, 263, 264, 265, 268, 275, 276, or 279.
    Drops unnecessary id columns and any duplicated columns.    
    Creates a has_ool column where poolcnt = 1.
    Creates a has_basement column where basementsqft > 0.
    Replaces nan values with 'none' for heatingorsystemdesc
    Replaces nan values with 0 for hashottuborspa. This was done because the only values were nan or 1
    Drops calculatedbathnbr, fullbathcnt,  columns 
    Drops columns where the percentage of missing data is over 40%
    Drop rows where the percentage of missing data is over 50%
    '''
    df = acquire.load_zillow_data()
    df = df[df.propertylandusetypeid.isin([260, 261, 262, 263, 264, 265, 268, 275, 276, 279])]
    df.drop(columns = ['buildingclasstypeid', 'typeconstructiontypeid', 'storytypeid',
                   'propertylandusetypeid', 'heatingorsystemtypeid', 'architecturalstyletypeid',
                   'airconditioningtypeid', 'id', 'parcelid' , 'unitcnt', 'propertyzoningdesc', 
                   'finishedsquarefeet12', 'calculatedbathnbr', 'finishedsquarefeet12', 'fullbathcnt'], inplace = True)
    df.drop(columns = [c for c in df.columns if c.endswith('.1')], inplace=True)
    df['has_pool'] = df['poolcnt'] == 1
    df['has_basement'] = df['basementsqft'] > 0
    df.heatingorsystemdesc.fillna('none', inplace = True)
    df.hashottuborspa.fillna(0, inplace = True)
    df.replace({True:1, False:0}, inplace = True)
    df = handle_missing_values(df, .6, .5)
    return df

In [16]:
def handle_missing_values(df, prop_required_column=.5, prop_required_row=.5):
    '''
    This function takes in a DataFrame (df), a minium for prop_required_column [0:1] with deafult of .5, 
    and a minimum value [0:1] for prop_required_row with a default of .5.
    
    It will first drop columns who's missing data is less than prop_required_column value.
    It will then drop the rows who's missing data is lower than the prop_required_row.
    '''
    columns_to_keep = []
    columns = df.columns
    for col in columns:   
        if df[col].notnull().sum()/df.shape[0] >= prop_required_column:
            columns_to_keep.append(col)
    df = df[columns_to_keep]
    df['row_to_keep'] = df.notnull().sum(axis=1)/df.shape[1] >= prop_required_row
    df = df.loc[df['row_to_keep'] == True]
    df.drop(columns= ['row_to_keep'], inplace = True)
    return df

In [17]:
zillow = prep_zillow()
zillow.shape

C:\Users\anthony\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3417: DtypeWarning: Columns (69) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
<ipython-input-16-38dc43d33f77>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['row_to_keep'] = df.notnull().sum(axis=1)/df.shape[1] >= prop_required_row


(52828, 28)

In [18]:
zillow.isnull().sum().sort_values(ascending=False)

buildingqualitytypeid           19106
regionidcity                     1036
lotsizesquarefeet                 408
yearbuilt                         159
calculatedfinishedsquarefeet      127
censustractandblock               125
structuretaxvaluedollarcnt         93
regionidzip                        27
taxamount                           4
landtaxvaluedollarcnt               2
taxvaluedollarcnt                   1
propertycountylandusecode           0
hashottuborspa                      0
fips                                0
latitude                            0
longitude                           0
bedroomcnt                          0
has_basement                        0
rawcensustractandblock              0
regionidcounty                      0
has_pool                            0
roomcnt                             0
assessmentyear                      0
transactiondate                     0
logerror                            0
heatingorsystemdesc                 0
propertyland

In [19]:
print(zillow['buildingqualitytypeid'].describe())
print(zillow['buildingqualitytypeid'].median())
print(zillow['buildingqualitytypeid'].mode())
print(zillow['buildingqualitytypeid'].mean())


count    33722.000000
mean         6.266947
std          1.716054
min          1.000000
25%          5.000000
50%          6.000000
75%          8.000000
max         12.000000
Name: buildingqualitytypeid, dtype: float64
6.0
0    6.0
dtype: float64
6.266947393393037


In [20]:
zillow['fips'].value_counts()

6037.0    33921
6059.0    14121
6111.0     4786
Name: fips, dtype: int64

In [21]:
zillow['regionidcity'].value_counts()

12447.0    11439
5534.0      1808
40227.0     1499
46298.0     1426
16764.0     1085
           ...  
32927.0        3
31134.0        2
10815.0        1
36078.0        1
21395.0        1
Name: regionidcity, Length: 175, dtype: int64

In [22]:
zillow['regionidcounty'].value_counts()

3101.0    33921
1286.0    14121
2061.0     4786
Name: regionidcounty, dtype: int64

In [23]:
zillow[['regionidcounty', 'regionidzip']].loc[zillow['regionidzip'].isnull()]

,regionidcounty,regionidzip
4161,3101.0,NaN
9732,3101.0,NaN
10063,3101.0,NaN
13128,3101.0,NaN
13386,3101.0,NaN
18423,2061.0,NaN
26735,1286.0,NaN
30920,2061.0,NaN
34551,2061.0,NaN
34894,2061.0,NaN


In [24]:
from sklearn.model_selection import train_test_split
train_validate, test = train_test_split(zillow, test_size=.2, random_state=333)
train, validate = train_test_split(train_validate, test_size=.25, random_state=333)

# Finds the median value for numeric columns and imputes that value for all missing values
med_cols = [
"taxvaluedollarcnt",
"landtaxvaluedollarcnt",
"taxamount",
"structuretaxvaluedollarcnt",
"calculatedfinishedsquarefeet",
"lotsizesquarefeet",
"buildingqualitytypeid"
]

for col in med_cols:
    median = train[col].median()
    train[col].fillna(median, inplace=True)
    validate[col].fillna(median, inplace=True)
    test[col].fillna(median, inplace=True)

# Finds the mode for year and imputes that value for all missing values
mode = int(train['yearbuilt'].mode())# I had some friction when this returned a float (and there were no decimals anyways)
train['yearbuilt'].fillna(value=mode, inplace=True)
validate['yearbuilt'].fillna(value=mode, inplace=True)
test['yearbuilt'].fillna(value=mode, inplace=True)

C:\Users\anthony\anaconda3\lib\site-packages\pandas\core\series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


Imputing missing region city id

I have a feelign that regioncity id is a important feature to have and, rather than gettin rid of over 1k additional rows, I'm going to figure out a way to approximate which city id belongs to which zips code, which is not missing nearly as many values. To do this I will find the zip codes associated with missing cityids, excluding nan, and then find the mode of the city id assoicated with those values. This will be used to impute the values manually.

In [25]:
# Get a list of zip values from the train set and remove the nan value
zips = train[['regionidcity', 'censustractandblock','regionidzip']].loc[train['regionidcity'].isnull()].regionidzip.unique()
zips

array([96989., 97023., 96973., 96403.,    nan, 96373., 96292., 96346.,
       96993., 96377., 96295., 96395., 96494., 96956., 96962., 96244.,
       96488., 96186., 96040., 97065., 96975., 96990., 97048., 96337.,
       96492., 96969., 96270., 96982., 96489., 96221., 97083., 96044.,
       96374., 97047., 96505., 96284., 96203., 96398., 97026., 97081.,
       97008., 96342., 96387., 97003., 96368., 96273., 96959., 96247.,
       96126., 97005., 96507., 97016., 96242., 97043., 96352., 96174.,
       97027., 96169., 96225., 96110.])

In [26]:
# Finds most fequent regionidcity for each zipcode in the train data set
findcity = train[['regionidcity', 'regionidzip']].loc[(train['regionidzip'].isin(zips))]
findcity['regionidcity'].unique()

array([ 12447.,  24832.,     nan,  24435.,  18874.,  13693.,  22827.,
        20008.,  16764.,  34543.,  33252.,  34780.,  39308.,  27491.,
        32923.,  54311.,  47568.,  14634.,  10734.,  32380.,  46298.,
        24384.,  47019.,  24812.,  38032.,   8384.,  39306.,  37688.,
        39076.,  42967., 118217.,  18875., 118694.,  45602.,  52842.,
        40081.,  17597.,  54970.,  15554.,  40009., 396556., 113576.,
        54352.,  17882.,  17150., 118895.,  14542., 118994.,  16677.,
        56780.,  12773.,  38980.,  47762.,  42091.,  30187., 114834.,
        14906.,  47547.,  25271.,  50749.,  17686.,  12292.,  53571.,
        45457.,  32927.,  44833.,  24174.,  21778.,  30908.])

In [27]:
pairs = pd.DataFrame(findcity.groupby('regionidzip')['regionidcity'].agg(pd.Series.mode)).reset_index()
pairs.columns = ['zips', 'most_regionid']
pairs = pairs[pairs['zips'] != 96395.0]
pairs

,zips,most_regionid
0,96040.0,12447
1,96044.0,17882
2,96110.0,39076
3,96126.0,56780
4,96169.0,14634
5,96174.0,396556
6,96186.0,18874
7,96203.0,42967
8,96221.0,12447
9,96225.0,54352


In [28]:
train = train.merge(pairs, how='left', left_on='regionidzip', right_on='zips')
validate = validate.merge(pairs, how='left', left_on='regionidzip', right_on='zips')
test = test.merge(pairs, how='left', left_on='regionidzip', right_on='zips')

train

,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedfinishedsquarefeet,fips,hashottuborspa,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,...,taxamount,censustractandblock,transactiondate,logerror,heatingorsystemdesc,propertylandusedesc,has_pool,has_basement,zips,most_regionid
0,2.0,3.0,6.0,1625.0,6059.0,0.0,33897389.0,-117850495.0,9000.0,122,...,7055.86,6.059012e+13,2017-07-27,0.012724,none,Single Family Residential,0,0,NaN,NaN
1,4.0,4.0,10.0,2787.0,6037.0,0.0,33773839.0,-118423542.0,10780.0,0101,...,17711.24,6.037670e+13,2017-01-31,0.014283,Central,Single Family Residential,1,0,NaN,NaN
2,1.0,3.0,4.0,1008.0,6037.0,0.0,33910366.0,-118066092.0,4889.0,0100,...,3769.08,6.037552e+13,2017-08-16,0.008006,Floor/Wall,Single Family Residential,0,0,NaN,NaN
3,2.0,3.0,8.0,1975.0,6037.0,0.0,34062509.0,-118367463.0,8201.0,0100,...,16380.14,6.037216e+13,2017-05-10,0.093608,Central,Single Family Residential,0,0,NaN,NaN
4,2.0,3.0,6.0,1706.0,6037.0,0.0,34207687.0,-118563785.0,6432.0,0100,...,6091.72,6.037135e+13,2017-06-28,0.000468,Central,Single Family Residential,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31691,1.0,2.0,6.0,1561.0,6059.0,0.0,33884414.0,-117917540.0,5760.0,122,...,7019.22,6.059011e+13,2017-03-22,-0.156084,none,Single Family Residential,0,0,NaN,NaN
31692,2.5,4.0,6.0,2108.0,6059.0,0.0,33677679.0,-117976645.0,6065.0,122,...,7314.04,6.059099e+13,2017-03-21,-0.031871,none,Single Family Residential,1,0,NaN,NaN
31693,1.0,3.0,4.0,1504.0,6037.0,0.0,34118228.0,-117914579.0,10340.0,0100,...,2330.82,6.037404e+13,2017-08-22,-0.005794,Floor/Wall,Single Family Residential,0,0,NaN,NaN
31694,2.0,2.0,8.0,1826.0,6037.0,0.0,34156014.0,-118354804.0,10078.0,0101,...,11061.20,6.037126e+13,2017-07-06,-0.034088,Central,Single Family Residential,1,0,NaN,NaN


In [29]:
train.regionidcity.fillna(train.most_regionid, inplace=True)
validate.regionidcity.fillna(validate.most_regionid, inplace=True)
test.regionidcity.fillna(test.most_regionid, inplace=True)

In [30]:
train['regionidcity'].unique()

array([ 26531.,  33311.,  53636.,  12447.,  24832.,  54212.,  40227.,
        25459.,  10723.,   4406., 113576.,  54311.,  46298.,  17597.,
        53655.,  24435.,  10608.,  41673., 118225.,  45457.,  44116.,
        25218.,  24174.,  51239.,  18874.,  30908.,  33837.,  53571.,
        24812.,  24245.,  54722.,  21412.,  13693.,  22827.,  20008.,
        52650.,  13150.,  26964.,  27110.,  16764.,   5534.,  34543.,
        19177.,  50749.,  33252.,  27183.,  10774.,  47019.,  14542.,
        54053.,  45888.,  12520.,  37015.,  34780.,  39308.,  16677.,
        27491.,  33727.,  32923.,  12292.,  47568.,  38032.,  29189.,
        14634.,  47198.,  25458.,  10734.,  13091.,  32380.,  18098.,
        34278., 118875.,  17150.,  50677.,  10389.,  24384.,  42150.,
        26483.,  12773.,  10241., 396054.,   9840.,  51861.,   8384.,
        39306.,  51617.,  33836.,  25974.,  55753.,  11626.,  37688.,
        47762.,  39076.,  17686.,  15237.,  25953.,  26965.,  33612.,
        42967., 1182

In [31]:

# Drop the extra columns
train.drop(columns=['zips', 'most_regionid'], inplace=True)
validate.drop(columns=['zips', 'most_regionid'], inplace=True)
test.drop(columns=['zips', 'most_regionid'], inplace=True)



In [32]:
train

,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedfinishedsquarefeet,fips,hashottuborspa,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,...,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,transactiondate,logerror,heatingorsystemdesc,propertylandusedesc,has_pool,has_basement
0,2.0,3.0,6.0,1625.0,6059.0,0.0,33897389.0,-117850495.0,9000.0,122,...,2016.0,484060.0,7055.86,6.059012e+13,2017-07-27,0.012724,none,Single Family Residential,0,0
1,4.0,4.0,10.0,2787.0,6037.0,0.0,33773839.0,-118423542.0,10780.0,0101,...,2016.0,901208.0,17711.24,6.037670e+13,2017-01-31,0.014283,Central,Single Family Residential,1,0
2,1.0,3.0,4.0,1008.0,6037.0,0.0,33910366.0,-118066092.0,4889.0,0100,...,2016.0,186776.0,3769.08,6.037552e+13,2017-08-16,0.008006,Floor/Wall,Single Family Residential,0,0
3,2.0,3.0,8.0,1975.0,6037.0,0.0,34062509.0,-118367463.0,8201.0,0100,...,2016.0,1113559.0,16380.14,6.037216e+13,2017-05-10,0.093608,Central,Single Family Residential,0,0
4,2.0,3.0,6.0,1706.0,6037.0,0.0,34207687.0,-118563785.0,6432.0,0100,...,2016.0,394000.0,6091.72,6.037135e+13,2017-06-28,0.000468,Central,Single Family Residential,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31691,1.0,2.0,6.0,1561.0,6059.0,0.0,33884414.0,-117917540.0,5760.0,122,...,2016.0,538844.0,7019.22,6.059011e+13,2017-03-22,-0.156084,none,Single Family Residential,0,0
31692,2.5,4.0,6.0,2108.0,6059.0,0.0,33677679.0,-117976645.0,6065.0,122,...,2016.0,497348.0,7314.04,6.059099e+13,2017-03-21,-0.031871,none,Single Family Residential,1,0
31693,1.0,3.0,4.0,1504.0,6037.0,0.0,34118228.0,-117914579.0,10340.0,0100,...,2016.0,97949.0,2330.82,6.037404e+13,2017-08-22,-0.005794,Floor/Wall,Single Family Residential,0,0
31694,2.0,2.0,8.0,1826.0,6037.0,0.0,34156014.0,-118354804.0,10078.0,0101,...,2016.0,551779.0,11061.20,6.037126e+13,2017-07-06,-0.034088,Central,Single Family Residential,1,0


Additional columns to drop:
* unitcnt 
* propertyzoningdesc
* finishedsquarefeet12
* calculatedbathnbr
* finishedsquarefeet12
* fullbathcnt

Drop rows if full
* regionidzip - Due to having so few zips codes but properties exist across multiple counties
* 


In [ ]:
test = zillow.merge(pairs_dict, how='left', left_on='regionidzip', right_on='regionidzip')
test.isnull().sum()

In [ ]:
zillow.regionidcity.value_counts()

In [ ]:
test.regionidcity_x.value_counts()

In [ ]:
test['regionidcity_x'].fillna(test['regionidcity_y'], inplace=True)
test.isnull().sum()

In [ ]:
zillow['regionidcity'].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
train_validate, test = train_test_split(zillow, test_size=.2, random_state=333)
train, validate = train_test_split(train_validate, test_size=.25, random_state=333)


med_cols = [
"taxvaluedollarcnt",
"landtaxvaluedollarcnt",
"taxamount",
"structuretaxvaluedollarcnt",
"calculatedfinishedsquarefeet",
"lotsizesquarefeet",
"buildingqualitytypeid"
]

for col in med_cols:
    median = train[col].median()
    train[col].fillna(median, inplace=True)
    validate[col].fillna(median, inplace=True)
    test[col].fillna(median, inplace=True)

mode = int(train['yearbuilt'].mode())# I had some friction when this returned a float (and there were no decimals anyways)
train['yearbuilt'].fillna(value=mode, inplace=True)
validate['yearbuilt'].fillna(value=mode, inplace=True)
test['yearbuilt'].fillna(value=mode, inplace=True)
train.isnull().sum()

In [ ]:
# Get a list of zip values from the train set and remove the nan value
zips = train[['regionidcity', 'censustractandblock','regionidzip']].loc[train['regionidcity'].isnull()].regionidzip.unique()
zips = [96989., 97023., 96973., 96403., 96373., 96292., 96346.,
   96993., 96377., 96295., 96395., 96494., 96956., 96962., 96244.,
   96488., 96186., 96040., 97065., 96975., 96990., 97048., 96337.,
   96492., 96969., 96270., 96982., 96489., 96221., 97083., 96044.,
   96374., 97047., 96505., 96284., 96203., 96398., 97026., 97081.,
   97008., 96342., 96387., 97003., 96368., 96273., 96959., 96247.,
   96126., 97005., 96507., 97016., 96242., 97043., 96352., 96174.,
   97027., 96169., 96225., 96110.]

# Finds most fequent regionidcity for each zipcode in the train data set
findcity = train[['regionidcity', 'regionidzip']].loc[(train['regionidzip'].isin(zips))]

# Creates a data frame of those pairs and renames the columns
pairs = pd.DataFrame(findcity.groupby('regionidzip')['regionidcity'].agg(pd.Series.mode)).reset_index()
pairs.columns = ['zips', 'most_regionid']

# merges pairs data frame across the train, validate, and test data sets
train = train.merge(pairs, how='left', left_on='regionidzip', right_on='zips', copy=False)
validate = validate.merge(pairs, how='left', left_on='regionidzip', right_on='zips', copy=False)
test = test.merge(pairs, how='left', left_on='regionidzip', right_on='zips', copy=False)

# Fills missing regionidcity with the most_regionid_y values for each data set
train['regionidcity'].fillna('most_regionid_y', inplace=True)
validate['regionidcity'].fillna('most_regionid_y', inplace=True)
test['regionidcity'].fillna('most_regionid_y', inplace=True)
# Drop the extra columns
train.drop(columns=['zips', 'most_regionid'], inplace=True)
validate.drop(columns=['zips', 'most_regionid'], inplace=True)
test.drop(columns=['zips', 'most_regionid'], inplace=True)

In [ ]:
print(train.shape)
print(validate.shape)
print(test.shape)

In [ ]:
print(train.columns)
print(validate.columns)
print(test.columns)

In [ ]:
# Drop the extra columns
train.drop(columns=['zips', 'most_regionid'], inplace=True)
validate.drop(columns=['zips', 'most_regionid'], inplace=True)
test.drop(columns=['zips', 'most_regionid'], inplace=True)

# Returns the train, valiate, and test data frame.